# NYC taxi data regression

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](/sdk/resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Define different `CommandComponent` using YAML
- Create `Pipeline` load these components from YAML

**Motivations** - This notebook explains how to load component via SDK then use these components to build pipeline. We use NYC dataset, build pipeline with five steps, prep data, transform data, train model, predict results and evaluate model performance.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [2]:
#import required libraries
from azure.ml import MLClient, dsl
from azure.ml.entities import JobInput, JobOutput, load_component
from azure.ml._constants import AssetTypes
import piprunpkg

## 1.2. Configure credential

We are using `DefaultAzureCredential` to get access to workspace. When an access token is needed, it requests one using multiple identities(`EnvironmentCredential, ManagedIdentityCredential, SharedTokenCacheCredential, VisualStudioCodeCredential, AzureCliCredential, AzurePowerShellCredential`) in turn, stopping when one provides a token.
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for more information.

`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python) for all available credentials if it does not work for you.  

In [2]:
from azure.identity import DefaultAzureCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token('https://management.azure.com/.default')
except Exception as ex:
    # If exception happens when retrieve token, try exclude the failed credential like this then try again:
    # Exclude VSCode credential:
    # credential = DefaultAzureCredential(exclude_visual_studio_code_credential=True)
    raise Exception("Failed to retrieve a token from the included credentials due to the following exception, try to add `exclude_xxx_credential=True` to `DefaultAzureCredential` and try again.") from ex

EnvironmentCredential.get_token failed: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
ManagedIdentityCredential.get_token failed: ManagedIdentityCredential authentication unavailable, no managed identity endpoint found.
SharedTokenCacheCredential.get_token failed: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
VisualStudioCodeCredential.get_token failed: Failed to get Azure user details from Visual Studio Code.


## 1.3. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ml` to get a handle to the required Azure Machine Learning workspace. 

In [3]:
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": "<SUBSCRIPTION_ID>",
        "resource_group": "<RESOURCE_GROUP>",
        "workspace_name": "<WORKSPACE_NAME>"
    }

    if client_config["subscription_id"].startswith('<'):
        print("please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <WORKSPACE_NAME> in notebook cell")
        raise ex
    else:  # write and reload from config file
        import json, os
        config_path = "../../.azureml/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

Found the config file in: D:\practice\azureml-examples\sdk\jobs\.azureml\config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x0000020A27CFB3D0>,
         subscription_id=d511f82f-71ba-49a4-8233-d7be8a3650f4,
         resource_group_name=mire2etesting,
         workspace_name=gewa_ws)


## 1.4. Retrieve or create an Azure Machine Learning compute target

In [4]:
# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
try:
    ml_client.compute.get(name=cluster_name)
except Exception:
    print('Creating a new compute target...')
    from azure.ml.entities import AmlCompute
    compute = AmlCompute(
        name=cluster_name,
        size="Standard_D2_v2",
        max_instances=2
    )
    ml_client.compute.begin_create_or_update(compute)

# 2. Build pipeline

In [3]:
parent_dir = ''

# 1. Load components
prepare_data = load_component(
    yaml_file=parent_dir + "./prep.yml"
)
transform_data = load_component(
    yaml_file=parent_dir + "./transform.yml"
)
train_model = load_component(
    yaml_file=parent_dir + "./train.yml"
)
predict_result = load_component(
    yaml_file=parent_dir + "./predict.yml"
)
score_data = load_component(
    yaml_file=parent_dir + "./score.yml"
)

# 2. Construct pipeline
@dsl.pipeline(default_compute="cpu-cluster", default_datastore="workspaceblobstore")
def nyc_taxi_data_regression(pipeline_job_input):
    prepare_sample_data = prepare_data(raw_data=pipeline_job_input)
    transform_sample_data = transform_data(clean_data=prepare_sample_data.outputs.prep_data)
    train_with_sample_data = train_model(training_data=transform_sample_data.outputs.transformed_data)
    predict_with_sample_data = predict_result(
        model_input=train_with_sample_data.outputs.model_output, test_data=train_with_sample_data.outputs.test_data
    )
    score_with_sample_data = score_data(predictions=predict_with_sample_data.outputs.predictions, model=train_with_sample_data.outputs.model_output)
    return {
        "pipeline_job_prepped_data": prepare_sample_data.outputs.prep_data,
        "pipeline_job_transformed_data": transform_sample_data.outputs.transformed_data,
        "pipeline_job_trained_model": train_with_sample_data.outputs.model_output,
        "pipeline_job_test_data": train_with_sample_data.outputs.test_data,
        "pipeline_job_predictions": predict_with_sample_data.outputs.predictions,
        "pipeline_job_score_report": score_with_sample_data.outputs.score_report,
    }

pipeline = nyc_taxi_data_regression(
    JobInput(type=AssetTypes.URI_FOLDER, path=parent_dir + "./data/")
)
pipeline.outputs.pipeline_job_prepped_data = JobOutput(path="/prepped_data", mode="rw_mount")
pipeline.outputs.pipeline_job_transformed_data = JobOutput(path="/transformed_data", mode="rw_mount")
pipeline.outputs.pipeline_job_trained_model = JobOutput(path="/trained-model", mode="rw_mount")
pipeline.outputs.pipeline_job_test_data = JobOutput(path="/test_data", mode="rw_mount")
pipeline.outputs.pipeline_job_predictions = JobOutput(path="/predictions", mode="rw_mount")
pipeline.outputs.pipeline_job_score_report = JobOutput(path="/report", mode="rw_mount")

print(pipeline)

name: heroic_hat_33l3c4s8bd
display_name: nyc_taxi_data_regression
type: pipeline
inputs:
  pipeline_job_input:
    mode: ro_mount
    type: uri_folder
    path: azureml:./data/
outputs:
  pipeline_job_prepped_data:
    mode: rw_mount
    type: uri_folder
    path: /prepped_data
  pipeline_job_transformed_data:
    mode: rw_mount
    type: uri_folder
    path: /transformed_data
  pipeline_job_trained_model:
    mode: rw_mount
    type: uri_folder
    path: /trained-model
  pipeline_job_test_data:
    mode: rw_mount
    type: uri_folder
    path: /test_data
  pipeline_job_predictions:
    mode: rw_mount
    type: uri_folder
    path: /predictions
  pipeline_job_score_report:
    mode: rw_mount
    type: uri_folder
    path: /report
tags: {}
properties: {}
experiment_name: 2c_nyc_taxi_data_regression
compute: azureml:cpu-cluster
jobs:
  prepare_sample_data:
    $schema: '{}'
    type: command
    inputs:
      raw_data: ${{parent.inputs.pipeline_job_input}}
    outputs:
      prep_data: 

In [4]:
from piprunpkg import run_pipeline, CuratedEnvironmentDefaultRunMode
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
output_root_dir = "./local-run-output"
run_pipeline(output_root_dir=output_root_dir, pipeline_job=pipeline, curated_env_default_run_mode=CuratedEnvironmentDefaultRunMode.CONTAINER))

INFO:executor:############################################prepare_sample_data##############################################
INFO:executor:
INFO:executor:job prepare_sample_data finished with result: False
INFO:executor:Run job prepare_sample_data with failure return code: 1
INFO:executor:Traceback (most recent call last):
INFO:executor:  File "d:\practice\azureml-examples\sdk\jobs\pipelines\2c_nyc_taxi_data_regression\prep_src\prep.py", line 8, in <module>
INFO:executor:    from sklearn.linear_model import LinearRegression
INFO:executor:ModuleNotFoundError: No module named 'sklearn'
INFO:executor:
INFO:executor:pipeline heroic_hat_33l3c4s8bd stopped due to failed job prepare_sample_data!


## 3. Submit pipeline job

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(pipeline, experiment_name="pipeline_samples")
pipeline_job

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](/sdk/jobs/pipelines/)